# Basketball NEAR Results Analysis

This notebook analyzes and compares results from:
- ~Baseline NEAR experiments (from baseline_results.json)~
- Neurosym-lib NEAR experiments (from various results pickle files)
- Reported program from the NEAR paper

## Step 0: Data collection.

Compute results for all neurosym-lib experiments and the reported program
```bash
# Search for the top 40 programs on the Fly-v-Fly dataset
$ CUDA_VISIBLE_DEVICES=0 python notebooks/bball_reproduction/benchmark_bball.py

In [13]:
%load_ext autoreload
%autoreload 2
%cd /home/asehgal/neurosym-lib

import json
import pickle
from pathlib import Path

import numpy as np
import pandas as pd

from neurosym.examples.near.metrics import compute_metrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/asehgal/neurosym-lib


## Load Baseline Results

In [14]:
# baseline results not available as NEAR code
# doesn't include execution code for this.

# # Load best NEAR results
# near_results_pth = Path("outputs/bball_results/near_outputs/")

# baseline_results = {}
# for result_file in near_results_pth.rglob("test_results.json"):
#     print(f"Loading Baseline results from {result_file}")
#     with open(result_file, "r") as f:
#         expt_report = json.load(f)
#         experiment_name = result_file.parent.name
#         y_true = np.array(expt_report["true_vals"])
#         y_scores = np.array(expt_report["predicted_vals"])
#         expt_report["report"] = compute_metrics(y_scores, y_true)
#         baseline_results[experiment_name] = expt_report

## Load Neurosym-lib Results

In [15]:
file_path = "outputs/bball_results/reproduction.pkl"
neurosym_results = {}
if Path(file_path).exists():
    with open(file_path, "rb") as f:
        results = pickle.load(f)
        file_name = Path(file_path).stem
        for i, result in enumerate(results):
            key = f"neurosym_{file_name}_{i:03d}"
            neurosym_results[key] = result
    print(f"  Loaded {len(results)} programs from {file_name}")
else:
    print(f"Warning: {file_path} not found")

  Loaded 40 programs from reproduction


## Create Comparison Table

In [16]:
# Combine all results
all_results = {}
# all_results.update(baseline_results)
all_results.update(neurosym_results)

print(f"Total experiments: {len(all_results)}")

Total experiments: 40


In [17]:
# Create comparison dataframe
table_data = []

for name, result in all_results.items():
    # Extract metrics
    report = result.get("report", {})
    
    # Handle different report structures
    if isinstance(report, dict):
        if "report" in report:
            macro_avg = report["report"].get("macro avg", {})
            hamming_acc = report.get("hamming_accuracy", 0.0)
        elif "macro avg" in report:
            macro_avg = report["macro avg"]
            hamming_acc = report.get("hamming_accuracy", 0.0)
        else:
            # Need to compute metrics
            if "pred_vals" in result and "true_vals" in result:
                pred_vals = np.array(result["pred_vals"])
                true_vals = np.array(result["true_vals"])
                metrics = compute_metrics(pred_vals, true_vals)
                macro_avg = metrics["report"]["macro avg"]
                hamming_acc = metrics["hamming_accuracy"]
            else:
                continue
    else:
        continue
    
    row = {
        "experiment": name,
        "precision": macro_avg.get("precision", 0.0),
        "recall": macro_avg.get("recall", 0.0),
        "f1_score": macro_avg.get("f1-score", 0.0),
        "support": macro_avg.get("support", 0),
        "hamming_accuracy": hamming_acc,
        "time": result.get("time", 0.0),
    }
    table_data.append(row)

df = pd.DataFrame(table_data)

# Sort by f1_score descending
df = df.sort_values("hamming_accuracy", ascending=False)

print("\n" + "=" * 80)
print("RESULTS COMPARISON")
print("=" * 80)
print(df.to_string(index=False))
print("=" * 80)


RESULTS COMPARISON
               experiment  precision   recall  f1_score  support  hamming_accuracy        time
neurosym_reproduction_000   0.101749 0.239139  0.142236  67325.0          0.323713  241.053499
neurosym_reproduction_039   0.067619 0.163255  0.036533  67325.0          0.115084 1004.492519
neurosym_reproduction_033   0.067554 0.160154  0.037228  67325.0          0.113450  829.661858
neurosym_reproduction_027   0.065004 0.159230  0.037332  67325.0          0.112960  644.927882
neurosym_reproduction_021   0.067801 0.157262  0.037918  67325.0          0.111964  449.200706
neurosym_reproduction_015   0.067489 0.155220  0.038378  67325.0          0.110939  276.600315
neurosym_reproduction_034   0.057352 0.151551  0.043157  67325.0          0.107092  829.662319
neurosym_reproduction_028   0.056438 0.150414  0.043272  67325.0          0.106424  644.929842
neurosym_reproduction_022   0.054699 0.147773  0.043375  67325.0          0.104790  449.200884
neurosym_reproduction_016   0.

## Summary Statistics

In [18]:
print("\nSUMMARY STATISTICS")
print("=" * 80)
print(f"Total experiments: {len(df)}")
print(f"\nBest F1-score: {df['f1_score'].max():.6f}")
print(f"  Experiment: {df.loc[df['f1_score'].idxmax(), 'experiment']}")
print(f"\nBest Hamming accuracy: {df['hamming_accuracy'].max():.6f}")
print(f"  Experiment: {df.loc[df['hamming_accuracy'].idxmax(), 'experiment']}")
print(f"\nAverage F1-score: {df['f1_score'].mean():.6f}")
print(f"Average Hamming accuracy: {df['hamming_accuracy'].mean():.6f}")
print(f"Average training time: {df['time'].mean():.2f} seconds")

# Baseline vs Neurosym comparison
baseline_df = df[df['experiment'].str.contains('baseline|crim13_astar|crim13_iddfs|enumeration', case=False, na=False)]
neurosym_df = df[df['experiment'].str.contains('neurosym|reproduction', case=False, na=False)]

if not baseline_df.empty:
    print(f"\nBaseline experiments: {len(baseline_df)}")
    print(f"  Avg F1: {baseline_df['f1_score'].mean():.6f}")
    print(f"  Avg Hamming Acc: {baseline_df['hamming_accuracy'].mean():.6f}")
    
if not neurosym_df.empty:
    print(f"\nNeurosym-lib experiments: {len(neurosym_df)}")
    print(f"  Avg F1: {neurosym_df['f1_score'].mean():.6f}")
    print(f"  Avg Hamming Acc: {neurosym_df['hamming_accuracy'].mean():.6f}")

# Reported program
reported_df = df[df['experiment'] == 'reported_program']
if not reported_df.empty:
    print("\nReported Program:")
    print(f"  F1: {reported_df['f1_score'].values[0]:.6f}")
    print(f"  Hamming Acc: {reported_df['hamming_accuracy'].values[0]:.6f}")
    print(f"  Time: {reported_df['time'].values[0]:.2f}s")

print("=" * 80)


SUMMARY STATISTICS
Total experiments: 40

Best F1-score: 0.142236
  Experiment: neurosym_reproduction_000

Best Hamming accuracy: 0.323713
  Experiment: neurosym_reproduction_000

Average F1-score: 0.037786
Average Hamming accuracy: 0.092587
Average training time: 446.18 seconds

Neurosym-lib experiments: 40
  Avg F1: 0.037786
  Avg Hamming Acc: 0.092587


## Save Results

In [19]:
# Save CSV
csv_path = "outputs/bball_results/comparison.csv"
Path(csv_path).parent.mkdir(parents=True, exist_ok=True)
df.to_csv(csv_path, index=False)
print(f"Saved CSV to: {csv_path}")

# Save Markdown
md_path = "outputs/bball_results/comparison.md"
with open(md_path, "w") as f:
    f.write("# MICE-DSL NEAR Results Comparison\n\n")
    f.write(df.to_markdown(index=False))
    f.write("\n")
print(f"Saved Markdown to: {md_path}")

Saved CSV to: outputs/bball_results/comparison.csv
Saved Markdown to: outputs/bball_results/comparison.md


## Display Top Programs

In [20]:
# Show top 10 programs by F1 score
print("\nTop 10 Programs by F1-score:")
print("=" * 80)
top_10 = df.head(10)
print(top_10.to_string(index=False))

# Show program strings for top results if available
print("\nProgram Details:")
print("=" * 80)
for idx, row in top_10.iterrows():
    exp_name = row['experiment']
    if exp_name in all_results:
        result = all_results[exp_name]
        program_str = result.get('program_str', str(result.get('program', 'N/A')))
        print(f"\n{exp_name}:")
        print(f"  Program: {program_str}")
        print(f"  F1: {row['f1_score']:.6f}, Hamming Acc: {row['hamming_accuracy']:.6f}, Time: {row['time']:.2f}s")


Top 10 Programs by F1-score:
               experiment  precision   recall  f1_score  support  hamming_accuracy        time
neurosym_reproduction_000   0.101749 0.239139  0.142236  67325.0          0.323713  241.053499
neurosym_reproduction_039   0.067619 0.163255  0.036533  67325.0          0.115084 1004.492519
neurosym_reproduction_033   0.067554 0.160154  0.037228  67325.0          0.113450  829.661858
neurosym_reproduction_027   0.065004 0.159230  0.037332  67325.0          0.112960  644.927882
neurosym_reproduction_021   0.067801 0.157262  0.037918  67325.0          0.111964  449.200706
neurosym_reproduction_015   0.067489 0.155220  0.038378  67325.0          0.110939  276.600315
neurosym_reproduction_034   0.057352 0.151551  0.043157  67325.0          0.107092  829.662319
neurosym_reproduction_028   0.056438 0.150414  0.043272  67325.0          0.106424  644.929842
neurosym_reproduction_022   0.054699 0.147773  0.043375  67325.0          0.104790  449.200884
neurosym_reproductio